In [82]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

RANDOM_STATE = 0

## 1. Load data & basic setup

In [83]:
csv_path = "../data/airquality_clean.csv"

df = pd.read_csv(csv_path)

df["datetime"] = pd.to_datetime(df["datetime"])
df = df.set_index("datetime")

df = df.sort_index()

print("Data shape:", df.shape)
print("Time range:", df.index.min(), "->", df.index.max())

candidate_targets = ["C6H6(GT)", "CO(GT)", "NOx(GT)", "NO2(GT)"]

sensor_cols = ["PT08.S1(CO)", "PT08.S2(NMHC)", "PT08.S3(NOx)",
               "PT08.S4(NO2)", "PT08.S5(O3)"]

meteo_cols = ["T", "RH", "AH"]

calendar_cols = ["Hour", "Weekday", "Month"]

base_feature_cols = sensor_cols + meteo_cols + calendar_cols

print("Base feature columns:")
print(base_feature_cols)

Data shape: (9357, 16)
Time range: 2004-03-10 18:00:00 -> 2005-04-04 14:00:00
Base feature columns:
['PT08.S1(CO)', 'PT08.S2(NMHC)', 'PT08.S3(NOx)', 'PT08.S4(NO2)', 'PT08.S5(O3)', 'T', 'RH', 'AH', 'Hour', 'Weekday', 'Month']


# 2. Helper functions: lag and rolling features

In [84]:

def add_lag_features(data, cols, lags):
    for col in cols:
        if col not in data.columns:
            continue
        for lag in lags:
            new_name = f"{col}_lag{lag}"
            data[new_name] = data[col].shift(lag)
    return data


def add_rolling_features(data, cols, windows):
    for col in cols:
        if col not in data.columns:
            continue
        for win in windows:
            new_name = f"{col}_rollmean_{win}"
            data[new_name] = data[col].rolling(window=win, min_periods=1).mean()
    return data

# 3. Shared hourly temporal features for all targets

In [85]:
df_hourly_base = df.copy()

short_lags_h = [1, 3]       # short-term: 1h, 3h
long_lags_h  = [6, 24]      # long-term: 6h, 24h
all_lags_h   = short_lags_h + long_lags_h

df_hourly_base = add_lag_features(df_hourly_base, meteo_cols, all_lags_h)

print("df_hourly_base shape (with shared meteo lags):", df_hourly_base.shape)

df_hourly_base shape (with shared meteo lags): (9357, 28)


# 4. Train Hourly Models per Target and Export Feature Sets

In [86]:
df_hourly_master = df_hourly_base.copy()
hourly_results = []

for target_col in candidate_targets:
    print("\n" + "=" * 70)
    print(f"[Hourly] Running model for target: {target_col}")
    print("=" * 70)

    df_hourly_master = add_lag_features(df_hourly_master, [target_col], all_lags_h)

    short_windows_h = [3]   # 3-hour moving average
    long_windows_h  = [24]  # 24-hour moving average
    all_windows_h   = short_windows_h + long_windows_h

    df_hourly_master = add_rolling_features(df_hourly_master, [target_col], all_windows_h)

    meteo_lag_cols_h = [
        c for c in df_hourly_master.columns
        if any(c.startswith(m + "_lag") for m in meteo_cols)
    ]

    target_lag_cols_h = [
        c for c in df_hourly_master.columns
        if c.startswith(target_col + "_lag")
    ]
    target_roll_cols_h = [
        c for c in df_hourly_master.columns
        if c.startswith(target_col + "_rollmean_")
    ]

    feature_cols_hourly = (
        base_feature_cols
        + meteo_lag_cols_h
        + target_lag_cols_h
        + target_roll_cols_h
    )

    feature_cols_hourly = [c for c in feature_cols_hourly if c in df_hourly_master.columns]

    print("[Hourly] #features:", len(feature_cols_hourly))

    df_hourly_ml = df_hourly_master.dropna(subset=[target_col] + feature_cols_hourly)
    print("[Hourly] ML data shape:", df_hourly_ml.shape)

    if df_hourly_ml.empty:
        print(f"[Hourly] Warning: no valid rows for target {target_col}, skip.")
        continue

    # =======================
    # 2004 train 2005 test
    # =======================
    train_mask_h = df_hourly_ml.index.year == 2004
    test_mask_h  = df_hourly_ml.index.year == 2005

    X_train_h = df_hourly_ml.loc[train_mask_h, feature_cols_hourly]
    y_train_h = df_hourly_ml.loc[train_mask_h, target_col]

    X_test_h = df_hourly_ml.loc[test_mask_h, feature_cols_hourly]
    y_test_h = df_hourly_ml.loc[test_mask_h, target_col]

    print("[Hourly] train size:", X_train_h.shape, "test size:", X_test_h.shape)

    if len(X_train_h) == 0 or len(X_test_h) == 0:
        print(f"[Hourly] Warning: train/test too small for target {target_col}, skip.")
        continue

    rf_hourly = RandomForestRegressor(
        n_estimators=300,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    rf_hourly.fit(X_train_h, y_train_h)

    y_pred_train_h = rf_hourly.predict(X_train_h)
    y_pred_test_h  = rf_hourly.predict(X_test_h)

    rmse_train_h = np.sqrt(mean_squared_error(y_train_h, y_pred_train_h))
    rmse_test_h  = np.sqrt(mean_squared_error(y_test_h,  y_pred_test_h))

    print("\n===== Hourly model performance =====")
    
    print("RMSE (train, hourly):", rmse_train_h)
    print("RMSE (test, hourly): ", rmse_test_h)

    short_name = (
        target_col.replace("(GT)", "")
                  .replace("(", "")
                  .replace(")", "")
                  .replace("/", "_")
    )
    out_dir = f"../data/temporal_features/features_hourly_{short_name}.csv"
    df_hourly_ml.to_csv(out_dir)
    print(f"[Hourly] Saved to {out_dir}")

    hourly_results.append({
        "Target": target_col,
        "RMSE_train_hourly": rmse_train_h,
        "RMSE_test_hourly": rmse_test_h,
        "n_train_hourly": X_train_h.shape[0],
        "n_test_hourly": X_test_h.shape[0],
        "n_features_hourly": len(feature_cols_hourly),
    })

hourly_results_df = pd.DataFrame(hourly_results)
display(hourly_results_df)

hourly_master_path = "../data/temporal_features/features_hourly_master_all_targets.csv"
df_hourly_master.to_csv(hourly_master_path)
print(f"[Master] Saved hourly master features to {hourly_master_path}")


[Hourly] Running model for target: C6H6(GT)
[Hourly] #features: 29
[Hourly] ML data shape: (8858, 34)
[Hourly] train size: (6787, 29) test size: (2071, 29)

===== Hourly model performance =====
RMSE (train, hourly): 0.07207365440675842
RMSE (test, hourly):  0.060750980970943336
[Hourly] Saved to ../data/temporal_features/features_hourly_C6H6.csv

[Hourly] Running model for target: CO(GT)
[Hourly] #features: 29
[Hourly] ML data shape: (7037, 40)
[Hourly] train size: (5054, 29) test size: (1983, 29)

===== Hourly model performance =====
RMSE (train, hourly): 0.10736089462047148
RMSE (test, hourly):  0.39525094705102237
[Hourly] Saved to ../data/temporal_features/features_hourly_CO.csv

[Hourly] Running model for target: NOx(GT)
[Hourly] #features: 29
[Hourly] ML data shape: (7280, 46)
[Hourly] train size: (5231, 29) test size: (2049, 29)

===== Hourly model performance =====
RMSE (train, hourly): 14.293319658651667
RMSE (test, hourly):  49.634789193499344
[Hourly] Saved to ../data/tempo

,Target,RMSE_train_hourly,RMSE_test_hourly,n_train_hourly,n_test_hourly,n_features_hourly
0,C6H6(GT),0.072074,0.060751,6787,2071,29
1,CO(GT),0.107361,0.395251,5054,1983,29
2,NOx(GT),14.293320,49.634789,5231,2049,29
3,NO2(GT),3.660235,17.139136,5229,2049,29


[Master] Saved hourly master features to ../data/temporal_features/features_hourly_master_all_targets.csv


# 5. Shared Daily temporal features for all targets

In [87]:
df_daily_base = df.resample("D").mean()
base_feature_cols_daily = sensor_cols + meteo_cols + ["Weekday", "Month"]
print("Daily base shape (before FE):", df_daily_base.shape)

# Daily lags
short_lags_d = [1, 2]
long_lags_d  = [7]
all_lags_d   = short_lags_d + long_lags_d

df_daily_base = add_lag_features(df_daily_base, meteo_cols, all_lags_d)
print("df_daily_base shape (with shared meteo features):", df_daily_base.shape)

Daily base shape (before FE): (391, 16)
df_daily_base shape (with shared meteo features): (391, 25)


# 6. Train Daily Models per Target and Export Feature Sets

In [88]:
df_daily_master = df_daily_base.copy()
daily_results = []

for target_col in candidate_targets:
    print("\n" + "=" * 70)
    print(f"[Daily] Running model for target: {target_col}")
    print("=" * 70)

    df_daily_master[target_col] = df[target_col].resample("D").mean()

    df_daily_master = add_lag_features(df_daily_master, [target_col], all_lags_d)

    short_windows_d = [3]   # 3-day moving average
    long_windows_d  = [7]   # 7-day moving average
    all_windows_d   = short_windows_d + long_windows_d

    df_daily_master = add_rolling_features(df_daily_master, [target_col], all_windows_d)

    meteo_lag_cols_d = [
        c for c in df_daily_master.columns
        if any(c.startswith(m + "_lag") for m in meteo_cols)
    ]


    target_lag_cols_d = [
        c for c in df_daily_master.columns
        if c.startswith(target_col + "_lag")
    ]

    target_roll_cols_d = [
        c for c in df_daily_master.columns
        if c.startswith(target_col + "_rollmean_")
    ]

    feature_cols_daily = (
        base_feature_cols_daily +
        meteo_lag_cols_d +
        target_lag_cols_d +
        target_roll_cols_d
    )

    feature_cols_daily = [c for c in feature_cols_daily if c in df_daily_master.columns]

    print("[Daily] #features:", len(feature_cols_daily))

    df_daily_ml = df_daily_master.dropna(subset=[target_col] + feature_cols_daily)
    print("[Daily] ML data shape:", df_daily_ml.shape)

    if df_daily_ml.empty:
        print(f"[Daily] Warning: no valid rows for {target_col}, skip.")
        continue

    # ========================
    # 2004 train / 2005 test
    # ========================
    train_mask_d = df_daily_ml.index.year == 2004
    test_mask_d  = df_daily_ml.index.year == 2005

    X_train_d = df_daily_ml.loc[train_mask_d, feature_cols_daily]
    y_train_d = df_daily_ml.loc[train_mask_d, target_col]

    X_test_d = df_daily_ml.loc[test_mask_d, feature_cols_daily]
    y_test_d = df_daily_ml.loc[test_mask_d, target_col]

    print("[Daily] train size:", X_train_d.shape, "test size:", X_test_d.shape)

    if len(X_train_d) == 0 or len(X_test_d) == 0:
        print(f"[Daily] Warning: insufficient data for {target_col}, skip.")
        continue

    rf_daily = RandomForestRegressor(
        n_estimators=300,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )

    rf_daily.fit(X_train_d, y_train_d)

    y_pred_train_d = rf_daily.predict(X_train_d)
    y_pred_test_d  = rf_daily.predict(X_test_d)

    rmse_train_d = np.sqrt(mean_squared_error(y_train_d, y_pred_train_d))
    rmse_test_d  = np.sqrt(mean_squared_error(y_test_d,  y_pred_test_d))

    print("\n===== Daily model performance =====")
    print("RMSE (train, daily):", rmse_train_d)
    print("RMSE (test, daily): ", rmse_test_d)

    short_name = (
        target_col.replace("(GT)", "")
                  .replace("(", "")
                  .replace(")", "")
                  .replace("/", "_")
    )
    out_dir = f"../data/temporal_features/features_daily_{short_name}.csv"
    df_daily_ml.to_csv(out_dir)
    print(f"[Daily] Saved to {out_dir}")

    daily_results.append({
        "Target": target_col,
        "RMSE_train_daily": rmse_train_d,
        "RMSE_test_daily": rmse_test_d,
        "n_train_daily": X_train_d.shape[0],
        "n_test_daily": X_test_d.shape[0],
        "n_features_daily": len(feature_cols_daily),
    })

# --- 小结结果 ---
daily_results_df = pd.DataFrame(daily_results)
display(daily_results_df)

daily_master_path = "../data/temporal_features/features_daily_master_all_targets.csv"
df_daily_master.to_csv(daily_master_path)
print(f"[Master] Saved daily master features to {daily_master_path}")


[Daily] Running model for target: C6H6(GT)
[Daily] #features: 24
[Daily] ML data shape: (360, 30)
[Daily] train size: (276, 24) test size: (84, 24)

===== Daily model performance =====
RMSE (train, daily): 0.16044107825269777
RMSE (test, daily):  0.3681122120210432
[Daily] Saved to ../data/temporal_features/features_daily_C6H6.csv

[Daily] Running model for target: CO(GT)
[Daily] #features: 24
[Daily] ML data shape: (273, 35)
[Daily] train size: (191, 24) test size: (82, 24)

===== Daily model performance =====
RMSE (train, daily): 0.13563099710460702
RMSE (test, daily):  0.49072560296256185
[Daily] Saved to ../data/temporal_features/features_daily_CO.csv

[Daily] Running model for target: NOx(GT)
[Daily] #features: 24
[Daily] ML data shape: (279, 40)
[Daily] train size: (197, 24) test size: (82, 24)

===== Daily model performance =====
RMSE (train, daily): 15.530831829607
RMSE (test, daily):  45.12683845173671
[Daily] Saved to ../data/temporal_features/features_daily_NOx.csv

[Daily]

,Target,RMSE_train_daily,RMSE_test_daily,n_train_daily,n_test_daily,n_features_daily
0,C6H6(GT),0.160441,0.368112,276,84,24
1,CO(GT),0.135631,0.490726,191,82,24
2,NOx(GT),15.530832,45.126838,197,82,24
3,NO2(GT),4.033547,16.678987,197,82,24


[Master] Saved daily master features to ../data/temporal_features/features_daily_master_all_targets.csv


# 7. Comparison: hourly vs daily granularity

In [89]:
hourly_results_df = pd.DataFrame(hourly_results)
daily_results_df  = pd.DataFrame(daily_results)

comparison_df = pd.merge(
    hourly_results_df,
    daily_results_df,
    on="Target",
    how="inner"
)

comparison_df["Delta_RMSE_test"] = (
    comparison_df["RMSE_test_daily"] - comparison_df["RMSE_test_hourly"]
)
comparison_df["Relative_Improvement_%"] = (
    100 * (comparison_df["RMSE_test_daily"] - comparison_df["RMSE_test_hourly"])
    / comparison_df["RMSE_test_daily"]
)

display(comparison_df)

,Target,RMSE_train_hourly,RMSE_test_hourly,n_train_hourly,n_test_hourly,n_features_hourly,RMSE_train_daily,RMSE_test_daily,n_train_daily,n_test_daily,n_features_daily,Delta_RMSE_test,Relative_Improvement_%
0,C6H6(GT),0.072074,0.060751,6787,2071,29,0.160441,0.368112,276,84,24,0.307361,83.496614
1,CO(GT),0.107361,0.395251,5054,1983,29,0.135631,0.490726,191,82,24,0.095475,19.455813
2,NOx(GT),14.293320,49.634789,5231,2049,29,15.530832,45.126838,197,82,24,-4.507951,-9.989512
3,NO2(GT),3.660235,17.139136,5229,2049,29,4.033547,16.678987,197,82,24,-0.460150,-2.758859


Different pollutants exhibit different temporal behaviors.
Our comparison suggests that hourly granularity may substantially improve prediction accuracy for pollutants with rapid short-term fluctuations (C6H6, CO), possibly because their dynamics are strongly influenced by short-lived events such as traffic emissions and micro-meteorological variations. Hourly lagged and rolling features may therefore help capture these fast temporal dependencies more effectively.

In contrast, NOx and NO₂ appear to exhibit extremely noisy hour-to-hour behavior, perhaps due to photochemical reactions, atmospheric dispersion, and potential sensor instability. Daily averaging may reduce high-frequency noise, allowing the model to learn more stable long-term patterns. As a result, the daily models could outperform hourly models for these pollutants.

Overall, these observations imply that the optimal temporal granularity might be pollutant-specific, and selecting an appropriate time scale is likely important for achieving robust prediction performance.